#### Code to validate models

In [130]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [132]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from collections import Counter
from tqdm.notebook import tqdm

In [133]:
from params import *

In [134]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample

from utils.logger import Config

from utils.metrics import per_class_average_precision_score, study_level_map

## Data

### Load

In [135]:
EXP_FOLDERS = [
    LOG_PATH + "2021-07-25/10/",
    LOG_PATH + "2021-07-26/1/",
    LOG_PATH + "2021-07-26/2/",
]

EXP_FOLDER = EXP_FOLDERS[-1]

In [136]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [137]:
# metrics = pd.read_csv(EXP_FOLDER + "metrics.csv")
pred_oof_img = np.mean([np.load(f + "pred_oof_img.npy") for f in EXP_FOLDERS], 0)
pred_oof_study = np.mean([np.load(f + "pred_oof_study.npy") for f in EXP_FOLDERS], 0)

df = pd.read_csv(EXP_FOLDER + "data.csv")

In [138]:
FOLD_IDX = 0

In [139]:
weights = sorted([EXP_FOLDER + p for p in os.listdir(EXP_FOLDER) if 'pt' in p])

In [140]:
# model = get_model(
#     config.selected_model,
#     reduce_stride=config.reduce_stride,
#     num_classes=config.num_classes,
# ).to(config.device)
# model.zero_grad()

# model.eval()

# model = load_model_weights(model, weights[FOLD_IDX])

In [141]:
pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
df['pred_img'] = pred_oof_img
# df = df[df['negative_pred'] > 0]

In [142]:
groups = df[['study_id', 'pred_img']].groupby('study_id').mean().rename(
    columns={'pred_img': 'pred_img_merged'}
).reset_index()
df = df.merge(groups, on="study_id", how="left")

In [143]:
auc = roc_auc_score(df['img_target'], pred_oof_img)
print(f'Image AUC : {auc :.3f}')
acc = accuracy_score(df['img_target'], pred_oof_img > 0.5)
print(f'Image Acc : {acc :.3f}')

Image AUC : 0.874
Image Acc : 0.816


In [144]:
auc = roc_auc_score(df['img_target'], df['pred_img_merged'])
print(f'Image AUC : {auc :.3f}')
acc = accuracy_score(df['img_target'], df['pred_img_merged'] > 0.5)
print(f'Image Acc : {acc :.3f}')

Image AUC : 0.872
Image Acc : 0.811


In [93]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.3f}')

Study mAP : 0.376


In [387]:
pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
df['pred_img'] = pred_oof_img
# df = df[df['negative_pred'] > 0]

In [388]:
df_study = df[['study_id'] + pred_cols + CLASSES + ['pred_img']].groupby('study_id').agg(np.mean)

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study.loc[df_study['pred_img'] > 0.9, 'negative_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.1, 'negative_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'typical_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 2

per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)

array([0.78961698, 0.84612411, 0.29082112, 0.32929603])

In [300]:
CLASSES

['negative', 'typical', 'indeterminate', 'atypical']

In [398]:
df_study = df[['study_id'] + pred_cols + CLASSES + ['img_target', 'pred_img']].groupby('study_id').agg(np.mean).copy()

df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study['typical_pred'] *= df_study['pred_img'] 
# df_study['indeterminate_pred'] *= df_study['pred_img'] 
# df_study['atypical_pred'] *= df_study['pred_img'] 

# df_study.loc[df_study['pred_img'] > 0.9, 'negative_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.1, 'negative_pred'] *= 2
# df_study.loc[df_study['pred_img'] > 0.9, 'typical_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 1.5
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] < 0.2, 'typical_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'indeterminate_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'atypical_pred'] *= 0.9

accs = per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)
np.round(accs, 4), np.round(np.mean(accs) * 2/3, 3)

(array([0.7886, 0.8461, 0.2908, 0.3293]), 0.376)

In [200]:
df_study.head()

,negative_pred,typical_pred,indeterminate_pred,atypical_pred,negative,typical,indeterminate,atypical,img_target,pred_img
study_id,,,,,,,,,,
00086460a852,0.113821,0.357324,0.241017,0.287838,0.0,1.0,0.0,0.0,1.0,0.742756
000c9c05fd14,0.001350,0.000055,0.001169,0.997426,0.0,0.0,0.0,1.0,0.0,0.099493
00292f8c37bd,0.932226,0.019807,0.043060,0.004907,1.0,0.0,0.0,0.0,0.0,0.091525
005057b3f880,0.938680,0.006872,0.041675,0.012773,1.0,0.0,0.0,0.0,0.0,0.045715
0051d9b12e72,0.085531,0.538621,0.306945,0.068903,0.0,0.0,0.0,1.0,1.0,0.904129


In [179]:
df_study = df[['study_id'] + pred_cols + CLASSES + ['pred_img']].groupby('study_id').agg(np.mean)

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study.loc[df_study['pred_img'] > 0.9, 'negative_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.1, 'negative_pred'] *= 2
# df_study.loc[df_study['pred_img'] < 0.1, 'typical_pred'] *= 2
# df_study.loc[df_study['pred_img'] < 0.1, 'indeterminate_pred'] *= 2
# df_study.loc[df_study['pred_img'] < 0.1, 'atypical_pred'] *= 2

per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)

array([0.77049216, 0.83492985, 0.28255951, 0.31372431])

In [204]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.3f}')

Study mAP : 0.367


In [205]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values, agg=np.max)
print(f'Study mAP : {study_map :.3f}')

Study mAP : 0.366


In [206]:
per_class_average_precision_score(pred_oof_study, df[CLASSES].values, 4, average=False)

array([0.98434671, 0.84566455, 0.34956159, 0.23040858])

In [399]:
pred_oof_study[:, 0] *= (1 - df['img_target'])
pred_oof_study[:, 1] *= df['img_target']
pred_oof_study[:, 2] *= df['img_target']
pred_oof_study[:, 3] *= df['img_target']

In [400]:
per_class_average_precision_score(pred_oof_study, df[CLASSES].values, 4, average=False)

array([0.98557206, 0.85187214, 0.35610549, 0.23691164])

In [401]:
study_map = study_level_map(pred_oof_study, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.3f}')

Study mAP : 0.411


In [402]:
study_map = study_level_map(pred_oof_study, df[CLASSES].values, df['study_id'].values, agg=np.max)
print(f'Study mAP : {study_map :.3f}')

Study mAP : 0.411


## Results

In [ ]:
pred_oof_study = pd.read_csv('../output/OOF_study_only_EBV2M_768.csv')
pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])

df_study = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').agg(np.mean)
df_study = df_study.merge(pred_oof_study, how="left", left_on="study_id", right_on="id").dropna()

In [ ]:
def proc(x):
    x = x.split('0 0 1 1')[:4]
    x = [float(y.strip().split(' ')[1]) for y in x]
    return x

pred_oof_study = np.array(df_study['PredictionString'].apply(proc).values.tolist())
# pred_oof_study = np.random.random(pred_oof_study.shape)
df_study[pred_cols] = pred_oof_study

In [ ]:
per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
num_classes=NUM_CLASSES) * 2/3